# Permutation Importance on Model M036

In [43]:
import pandas as pd
import catboost
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
import numpy as np

In [56]:
def permutation_importance(model, X_val, y_val, metric, threshold=0.005,
                           minimize=True, verbose=True):
    results = {}
    
    y_pred = model.predict(X_val)
    
    results['base_score'] = metric(y_val, y_pred)
    if verbose:
        print(f'Base score {results["base_score"]:.5}')

    
    for col in tqdm(X_val.columns):
        freezed_col = X_val[col].copy()

        X_val[col] = np.random.permutation(X_val[col])
        preds = model.predict(X_val, thread_count=20)
        results[col] = metric(y_val, preds)

        X_val[col] = freezed_col
        
        if verbose:
            print(f'column: {col} - {results[col]:.5}')
    
    if minimize:
        bad_features = [k for k in results if results[k] < results['base_score'] + threshold]
    else:
        bad_features = [k for k in results if results[k] > results['base_score'] + threshold]
    bad_features.remove('base_score')
    
    return results, bad_features

In [4]:
model = catboost.CatBoostRegressor()

In [13]:
model.load_model('../models/M036/M036-0705_2317-1JHC-1.model')

In [14]:
df = pd.read_parquet('../data/FE014/FE014-train-1JHC.parquet')

In [22]:
train_df = pd.read_parquet('../data/FE008_train.parquet') # only loading for skeleton not features
mol_group = train_df[['molecule_name','type']].copy()

In [51]:
N_FOLDS = 2
folds = GroupKFold(n_splits=N_FOLDS)
bond_type = '1JHC'
X = df[model.feature_names_]
y = df['scalar_coupling_constant']
mol_group_type = mol_group.loc[mol_group['type'] == bond_type]['molecule_name']

for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X, groups=mol_group_type)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    X_valid = X_valid.copy()
    model = catboost.CatBoostRegressor()
    model.load_model('../models/M036/M036-0705_2317-1JHC-{}.model'.format(fold_n + 1))
    results, bad_features = permutation_importance(model, X_valid, y_valid, metric=mean_absolute_error)
    break



  0%|          | 0/178 [00:00<?, ?it/s]

Base score 0.61331




  1%|          | 1/178 [00:31<1:34:20, 31.98s/it]

column: atom1_valence - 1.776




  1%|          | 2/178 [01:06<1:36:07, 32.77s/it]

...

column: NN - 0.61412




 99%|█████████▉| 177/178 [1:33:30<00:31, 31.71s/it]

column: NF - 0.61332




100%|██████████| 178/178 [1:34:00<00:00, 31.37s/it]

column: NO - 0.61668


In [52]:
bad_features

['atom1_spin_multiplicity',
 'is_bond',
 'closest_to_0_atomic_mass',
 '2nd_closest_to_0_spin_multiplicity',
 '3rd_closest_to_0_spin_multiplicity',
 '4th_closest_to_0_spin_multiplicity',
 '5th_closest_to_0_spin_multiplicity',
 '6th_closest_to_0_spin_multiplicity',
 '7th_closest_to_0_spin_multiplicity',
 '8th_closest_to_0_spin_multiplicity',
 '9th_closest_to_0_spin_multiplicity',
 '10th_closest_to_0_atomic_mass',
 '10th_closest_to_0_spin_multiplicity',
 'closest_to_1_atomic_mass',
 'closest_to_1_valence',
 'closest_to_1_spin_multiplicity',
 '2nd_closest_to_1_atomic_mass',
 '2nd_closest_to_1_spin_multiplicity',
 '3rd_closest_to_1_spin_multiplicity',
 '4th_closest_to_1_spin_multiplicity',
 '5th_closest_to_1_spin_multiplicity',
 '6th_closest_to_1_spin_multiplicity',
 '7th_closest_to_1_spin_multiplicity',
 '8th_closest_to_1_spin_multiplicity',
 '9th_closest_to_1_spin_multiplicity',
 '10th_closest_to_1_atomic_mass',
 '10th_closest_to_1_spin_multiplicity',
 'num_atoms',
 'angle_clos_0_2nd',
 '

# 3JHC

In [57]:
folds = GroupKFold(n_splits=N_FOLDS)
bond_type = '3JHC'
N_FOLDS = 2
df = pd.read_parquet('../data/FE014/FE014-train-{}.parquet'.format(bond_type))
X = df[model.feature_names_]
y = df['scalar_coupling_constant']
mol_group_type = mol_group.loc[mol_group['type'] == bond_type]['molecule_name']

for fold_n, (train_idx, valid_idx) in enumerate(folds.split(X, groups=mol_group_type)):
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    X_valid = X_valid.copy()
    model = catboost.CatBoostRegressor()
    model.load_model('../models/M036/M036-0705_2317-{}-{}.model'.format(bond_type, fold_n + 1))
    results, bad_features = permutation_importance(model, X_valid, y_valid, metric=mean_absolute_error)
    break




  0%|          | 0/178 [00:00<?, ?it/s]

Base score 0.37588





  1%|          | 1/178 [00:56<2:47:24, 56.75s/it]

column: atom1_valence - 0.40058


KeyboardInterrupt: 